In [14]:
import pandas as pd 
import boto3
from io import StringIO
import numpy as np

In [15]:
session =boto3.session.Session()
s3 = session.client(
    service_name ='s3',
    endpoint_url ='https://storage.yandexcloud.net/',
    aws_access_key_id ='YCAJEwnMU9buPdj6Uk3NV5LmG',
    aws_secret_access_key ='YCPQy22uGBgPb-_pUgHRFx09fnfF-MapLWG2ojQx')

bucket = 'laboratory-ba'
file_name='mz_iris.csv'

response =s3.get_object(Bucket=bucket,Key=file_name)
data =response['Body'].read().decode('utf-8')

In [16]:
df = pd.read_csv(StringIO(data))

In [17]:
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
for column in df.columns:
    if df[column].dtype == 'float64' or df[column].dtype == 'int64':
        df[column] = df[column].fillna(df[column].median())
print(df.dtypes)
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    df = df[(df[col] >= (Q1 - 1.5 * IQR)) & (df[col] <= (Q3 + 1.5 * IQR))]

Id                 int64
SepalLengthCm    float64
SepalWidthCm     float64
PetalLengthCm    float64
PetalWidthCm     float64
Species           object
dtype: object


In [18]:
csv_buffer =StringIO()
df.to_csv(csv_buffer,index =False)
s3.put_object(Bucket=bucket,Key ='mz_iris_clean.csv', Body = csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '054af4e9eb0204b9',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'nginx',
   'date': 'Wed, 19 Mar 2025 09:06:52 GMT',
   'content-type': 'application/octet-stream',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'keep-alive': 'timeout=60',
   'etag': '"702acd11472138307ed78edfd935aeea"',
   'x-amz-expiration': 'expiry-date="Wed, 18 Jun 2025 00:00:00 GMT", rule-id="%D0%A3%D0%B4%D0%B0%D0%BB%D1%8F%D0%B5%D0%BC%20%D1%84%D0%B0%D0%B9%D0%BB%D1%8B%20%D1%81%D1%82%D0%B0%D1%80%D1%88%D0%B5%2090%20%D0%B4%D0%BD%D0%B5%D0%B9"',
   'x-amz-request-id': '054af4e9eb0204b9',
   'x-amz-storage-class': 'COLD'},
  'RetryAttempts': 0},
 'Expiration': 'expiry-date="Wed, 18 Jun 2025 00:00:00 GMT", rule-id="%D0%A3%D0%B4%D0%B0%D0%BB%D1%8F%D0%B5%D0%BC%20%D1%84%D0%B0%D0%B9%D0%BB%D1%8B%20%D1%81%D1%82%D0%B0%D1%80%D1%88%D0%B5%2090%20%D0%B4%D0%BD%D0%B5%D0%B9"',
 'ETag': '"702acd11472138307ed78edfd935aeea"'}